# OpenSearch Neural Sparse 모델 한국어 성능 개선 프로젝트

이 노트북은 OpenSearch의 neural sparse 모델(SPLADE 기반)의 한국어 성능을 개선하기 위한 재학습 파이프라인입니다.

## 주요 기능
1. 한국어 공개 데이터셋 수집 (Hugging Face)
2. 형태소 분석 (KoNLPy, Mecab)
3. 동의어 처리 및 트렌드 키워드 가중치 적용
4. SPLADE 모델 파인튜닝
5. OpenSearch 통합

## 1. 환경 설정 및 라이브러리 설치

In [ ]:
# 필수 라이브러리 설치
!pip install -q transformers datasets sentence-transformers
!pip install -q torch torchvision torchaudio
!pip install -q konlpy mecab-python3
!pip install -q openai anthropic  # 동의어 및 트렌드 분석용
!pip install -q scikit-learn pandas numpy matplotlib seaborn
!pip install -q accelerate huggingface-hub

# Mecab 형태소 분석기 설치 (Ubuntu/Debian)
!apt-get install -y g++ openjdk-8-jdk python3-dev
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [ ]:
import os
import json
import pandas as pd
import numpy as np
from datetime import datetime
from collections import defaultdict, Counter
import warnings
warnings.filterwarnings('ignore')

# Hugging Face & Transformers
from datasets import load_dataset, Dataset, DatasetDict
from transformers import (
    AutoTokenizer, 
    AutoModelForMaskedLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from sentence_transformers import SentenceTransformer

# 형태소 분석
from konlpy.tag import Mecab

# PyTorch
import torch
from torch import nn
from torch.utils.data import DataLoader

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

## 2. 한국어 데이터셋 수집

Hugging Face에서 다양한 한국어 공개 데이터셋을 수집합니다:
- **KLUE**: 한국어 이해 평가 벤치마크
- **KorQuAD**: 한국어 질의응답 데이터셋
- **Korean Wikipedia**: 한국어 위키피디아
- **NIKL**: 국립국어원 말뭉치

In [ ]:
def load_korean_datasets():
    """
    Hugging Face에서 한국어 공개 데이터셋을 로드합니다.
    """
    datasets_info = {}
    
    print("📚 한국어 데이터셋 로딩 중...\n")
    
    # 1. KLUE (Korean Language Understanding Evaluation)
    try:
        print("1️⃣ KLUE 데이터셋 로딩...")
        klue_tc = load_dataset("klue", "ynat", split="train")  # Topic Classification
        klue_sts = load_dataset("klue", "sts", split="train")  # Semantic Textual Similarity
        klue_mrc = load_dataset("klue", "mrc", split="train")  # Machine Reading Comprehension
        
        datasets_info['klue'] = {
            'topic_classification': klue_tc,
            'semantic_similarity': klue_sts,
            'reading_comprehension': klue_mrc
        }
        print(f"   ✓ Topic Classification: {len(klue_tc):,} samples")
        print(f"   ✓ Semantic Similarity: {len(klue_sts):,} samples")
        print(f"   ✓ Reading Comprehension: {len(klue_mrc):,} samples\n")
    except Exception as e:
        print(f"   ✗ KLUE 로딩 실패: {e}\n")
    
    # 2. KorQuAD (Korean Question Answering Dataset)
    try:
        print("2️⃣ KorQuAD 데이터셋 로딩...")
        korquad_v1 = load_dataset("squad_kor_v1", split="train")
        datasets_info['korquad'] = korquad_v1
        print(f"   ✓ KorQuAD v1: {len(korquad_v1):,} samples\n")
    except Exception as e:
        print(f"   ✗ KorQuAD 로딩 실패: {e}\n")
    
    # 3. Korean Wikipedia
    try:
        print("3️⃣ 한국어 Wikipedia 데이터셋 로딩...")
        ko_wiki = load_dataset("wikipedia", "20220301.ko", split="train[:50000]")  # 샘플링
        datasets_info['wikipedia'] = ko_wiki
        print(f"   ✓ Korean Wikipedia: {len(ko_wiki):,} samples\n")
    except Exception as e:
        print(f"   ✗ Wikipedia 로딩 실패: {e}\n")
    
    # 4. Korean Chat/Conversation datasets
    try:
        print("4️⃣ 한국어 대화 데이터셋 로딩...")
        korean_chat = load_dataset("heegyu/Korean-Culture-QA", split="train")
        datasets_info['korean_chat'] = korean_chat
        print(f"   ✓ Korean Culture QA: {len(korean_chat):,} samples\n")
    except Exception as e:
        print(f"   ✗ Korean Chat 로딩 실패: {e}\n")
    
    # 5. Korean News Dataset
    try:
        print("5️⃣ 한국어 뉴스 데이터셋 로딩...")
        korean_news = load_dataset("heegyu/news-category-dataset", split="train[:100000]")
        datasets_info['korean_news'] = korean_news
        print(f"   ✓ Korean News: {len(korean_news):,} samples\n")
    except Exception as e:
        print(f"   ✗ Korean News 로딩 실패: {e}\n")
    
    return datasets_info

# 데이터셋 로드
korean_datasets = load_korean_datasets()

print("\n" + "="*60)
print(f"총 {len(korean_datasets)}개의 데이터셋 카테고리가 로드되었습니다.")
print("="*60)

## 3. 데이터 전처리 및 텍스트 추출

In [ ]:
def extract_texts_from_datasets(datasets_info):
    """
    각 데이터셋에서 텍스트를 추출합니다.
    """
    all_texts = []
    
    # KLUE 데이터셋
    if 'klue' in datasets_info:
        klue_data = datasets_info['klue']
        
        # Topic Classification
        if 'topic_classification' in klue_data:
            texts = [item['title'] for item in klue_data['topic_classification']]
            all_texts.extend(texts)
        
        # Semantic Similarity
        if 'semantic_similarity' in klue_data:
            for item in klue_data['semantic_similarity']:
                all_texts.append(item['sentence1'])
                all_texts.append(item['sentence2'])
        
        # Reading Comprehension
        if 'reading_comprehension' in klue_data:
            for item in klue_data['reading_comprehension']:
                all_texts.append(item['context'])
                all_texts.append(item['question'])
    
    # KorQuAD
    if 'korquad' in datasets_info:
        for item in datasets_info['korquad']:
            all_texts.append(item['context'])
            all_texts.append(item['question'])
    
    # Wikipedia
    if 'wikipedia' in datasets_info:
        texts = [item['text'][:1000] for item in datasets_info['wikipedia']]  # 처음 1000자만
        all_texts.extend(texts)
    
    # Korean Chat
    if 'korean_chat' in datasets_info:
        for item in datasets_info['korean_chat']:
            if 'question' in item:
                all_texts.append(item['question'])
            if 'answer' in item:
                all_texts.append(item['answer'])
    
    # Korean News
    if 'korean_news' in datasets_info:
        for item in datasets_info['korean_news']:
            if 'headline' in item:
                all_texts.append(item['headline'])
    
    # 중복 제거 및 정제
    all_texts = list(set([text.strip() for text in all_texts if text and len(text.strip()) > 10]))
    
    print(f"총 {len(all_texts):,}개의 고유한 텍스트가 추출되었습니다.")
    return all_texts

# 텍스트 추출
corpus_texts = extract_texts_from_datasets(korean_datasets)

# 샘플 출력
print("\n샘플 텍스트:")
for i, text in enumerate(corpus_texts[:5], 1):
    print(f"{i}. {text[:100]}...")

## 4. 형태소 분석 및 키워드 추출

Mecab을 사용하여 한국어 형태소 분석을 수행하고 주요 키워드를 추출합니다.

In [ ]:
# Mecab 형태소 분석기 초기화
try:
    mecab = Mecab()
    print("✓ Mecab 형태소 분석기 초기화 완료")
except Exception as e:
    print(f"✗ Mecab 초기화 실패: {e}")
    print("대안으로 Okt를 사용합니다.")
    from konlpy.tag import Okt
    mecab = Okt()

def extract_morphemes(text, analyzer=mecab):
    """
    텍스트에서 형태소를 추출합니다.
    명사(NNG, NNP), 동사(VV), 형용사(VA)를 주로 추출합니다.
    """
    try:
        # 형태소 분석
        morphs = analyzer.pos(text)
        
        # 중요한 품사만 추출 (명사, 동사, 형용사)
        important_pos = ['NNG', 'NNP', 'VV', 'VA', 'XR', 'SL']  # 일반명사, 고유명사, 동사, 형용사, 어근, 외국어
        keywords = [word for word, pos in morphs if pos in important_pos and len(word) > 1]
        
        return keywords
    except Exception as e:
        return []

def analyze_corpus(texts, sample_size=10000):
    """
    코퍼스에서 형태소 분석을 수행하고 키워드 빈도를 계산합니다.
    """
    print(f"📊 {min(sample_size, len(texts)):,}개의 텍스트에서 형태소 분석 중...")
    
    all_keywords = []
    morpheme_data = []
    
    # 샘플링 (전체 처리는 시간이 오래 걸림)
    sample_texts = texts[:sample_size]
    
    for i, text in enumerate(sample_texts):
        if i % 1000 == 0:
            print(f"  진행: {i:,}/{len(sample_texts):,} ({i/len(sample_texts)*100:.1f}%)")
        
        keywords = extract_morphemes(text)
        all_keywords.extend(keywords)
        
        morpheme_data.append({
            'text': text,
            'keywords': keywords,
            'keyword_count': len(keywords)
        })
    
    # 키워드 빈도 계산
    keyword_freq = Counter(all_keywords)
    
    print(f"\n✓ 총 {len(all_keywords):,}개의 키워드 추출")
    print(f"✓ 고유 키워드 수: {len(keyword_freq):,}개")
    
    return morpheme_data, keyword_freq

# 형태소 분석 실행
morpheme_data, keyword_freq = analyze_corpus(corpus_texts)

# 상위 키워드 출력
print("\n🏆 상위 50개 키워드:")
for i, (keyword, freq) in enumerate(keyword_freq.most_common(50), 1):
    print(f"{i:2d}. {keyword:15s} - {freq:,}회")

## 5. 동의어 처리 및 트렌드 키워드 가중치

웹 검색을 통해 최신 트렌드 키워드를 식별하고, 동의어 사전을 구축합니다.

In [ ]:
# 한국어 동의어 사전 (기본)
BASE_SYNONYMS = {
    '인공지능': ['AI', '에이아이', '머신러닝', '딥러닝', '기계학습'],
    '검색': ['서치', 'search', '찾기', '탐색'],
    '데이터': ['자료', 'data', '정보'],
    '개발': ['개발하다', 'development', '제작', '구축'],
    '서비스': ['서비스하다', 'service', '제공'],
    '사용자': ['유저', 'user', '이용자', '고객'],
    '시스템': ['system', '체계', '시스템'],
    '기술': ['테크', 'tech', 'technology', '기법'],
    '분석': ['애널리틱스', 'analytics', '해석', '분석하다'],
    '모델': ['model', '모형'],
    '학습': ['training', '트레이닝', '러닝', 'learning'],
    '성능': ['퍼포먼스', 'performance', '효율'],
    '최적화': ['optimization', '개선', '향상'],
    '알고리즘': ['algorithm', '알고리즘'],
    '네트워크': ['network', '네트웍', '망'],
}

# 2024-2025 트렌드 키워드 (가중치 높음)
TREND_KEYWORDS = {
    'LLM': 3.0,  # Large Language Model
    'GPT': 3.0,
    'ChatGPT': 3.0,
    '챗GPT': 3.0,
    '생성형AI': 2.8,
    'GenAI': 2.8,
    '트랜스포머': 2.5,
    'Transformer': 2.5,
    'RAG': 2.7,  # Retrieval Augmented Generation
    '벡터검색': 2.6,
    '임베딩': 2.5,
    'embedding': 2.5,
    'BERT': 2.3,
    'OpenSearch': 2.4,
    'Elasticsearch': 2.2,
    '신경망': 2.3,
    '파인튜닝': 2.5,
    'fine-tuning': 2.5,
    'prompt': 2.7,
    '프롬프트': 2.7,
    'SPLADE': 2.8,
    '희소벡터': 2.6,
    'sparse': 2.5,
    '토큰': 2.4,
    'token': 2.4,
    'AI검색': 2.6,
    '시맨틱검색': 2.5,
    'semantic': 2.5,
}

def build_synonym_dict(base_synonyms, keyword_freq):
    """
    기본 동의어 사전과 키워드 빈도를 결합하여 확장된 동의어 사전을 만듭니다.
    """
    synonym_dict = {}
    
    for key, synonyms in base_synonyms.items():
        all_terms = [key] + synonyms
        for term in all_terms:
            synonym_dict[term.lower()] = all_terms
    
    return synonym_dict

def calculate_keyword_weights(keyword_freq, trend_keywords):
    """
    키워드의 가중치를 계산합니다.
    - 기본 빈도 기반 가중치
    - 트렌드 키워드 보너스 가중치
    """
    keyword_weights = {}
    
    # 빈도 정규화 (log scale)
    max_freq = max(keyword_freq.values()) if keyword_freq else 1
    
    for keyword, freq in keyword_freq.items():
        # 기본 가중치: 로그 스케일 빈도
        base_weight = np.log1p(freq) / np.log1p(max_freq)
        
        # 트렌드 보너스
        trend_bonus = 1.0
        for trend_key, bonus in trend_keywords.items():
            if trend_key.lower() in keyword.lower() or keyword.lower() in trend_key.lower():
                trend_bonus = max(trend_bonus, bonus)
        
        # 최종 가중치
        keyword_weights[keyword] = base_weight * trend_bonus
    
    return keyword_weights

# 동의어 사전 구축
synonym_dict = build_synonym_dict(BASE_SYNONYMS, keyword_freq)
print(f"✓ 동의어 사전 구축 완료: {len(synonym_dict):,}개 항목")

# 키워드 가중치 계산
keyword_weights = calculate_keyword_weights(keyword_freq, TREND_KEYWORDS)
print(f"✓ 키워드 가중치 계산 완료: {len(keyword_weights):,}개 키워드")

# 가중치 상위 키워드 출력
print("\n🔥 가중치 상위 30개 키워드:")
sorted_weights = sorted(keyword_weights.items(), key=lambda x: x[1], reverse=True)[:30]
for i, (keyword, weight) in enumerate(sorted_weights, 1):
    print(f"{i:2d}. {keyword:20s} - 가중치: {weight:.4f}")

## 6. 학습 데이터셋 준비

SPLADE 모델 학습을 위한 데이터셋을 준비합니다.

In [ ]:
def prepare_training_dataset(morpheme_data, keyword_weights, synonym_dict):
    """
    SPLADE 학습을 위한 데이터셋을 준비합니다.
    - Query-Document 쌍 생성
    - 동의어 확장
    - 가중치 적용
    """
    training_data = []
    
    for item in morpheme_data:
        text = item['text']
        keywords = item['keywords']
        
        # 키워드 기반 쿼리 생성
        if len(keywords) >= 2:
            # 상위 키워드 선택 (가중치 기반)
            weighted_keywords = [(kw, keyword_weights.get(kw, 0.5)) for kw in keywords]
            weighted_keywords.sort(key=lambda x: x[1], reverse=True)
            
            # 상위 3-5개 키워드로 쿼리 생성
            top_keywords = [kw for kw, _ in weighted_keywords[:np.random.randint(3, 6)]]
            query = ' '.join(top_keywords)
            
            # 동의어 확장된 쿼리도 추가
            expanded_queries = [query]
            for kw in top_keywords:
                if kw.lower() in synonym_dict:
                    synonyms = synonym_dict[kw.lower()]
                    if len(synonyms) > 1:
                        # 동의어 중 하나로 대체
                        alt_kw = np.random.choice([s for s in synonyms if s != kw])
                        alt_query = query.replace(kw, alt_kw)
                        expanded_queries.append(alt_query)
            
            # 학습 데이터 추가
            for q in expanded_queries:
                training_data.append({
                    'query': q,
                    'document': text,
                    'score': 1.0,  # Positive pair
                    'keywords': keywords
                })
    
    print(f"✓ 학습 데이터 생성 완료: {len(training_data):,}개 샘플")
    return training_data

# 학습 데이터셋 준비
training_data = prepare_training_dataset(morpheme_data, keyword_weights, synonym_dict)

# 데이터프레임으로 변환
train_df = pd.DataFrame(training_data)

print("\n📊 학습 데이터 샘플:")
print(train_df.head(10))

# 통계
print("\n📈 데이터셋 통계:")
print(f"총 샘플 수: {len(train_df):,}")
print(f"평균 쿼리 길이: {train_df['query'].str.len().mean():.1f}자")
print(f"평균 문서 길이: {train_df['document'].str.len().mean():.1f}자")

## 7. SPLADE 모델 초기화 및 설정

한국어 BERT 기반 SPLADE 모델을 초기화합니다.

In [ ]:
# 한국어 사전학습 모델 선택
MODEL_NAME = "klue/bert-base"  # 또는 "monologg/kobert", "beomi/kcbert-base"

# 토크나이저 및 모델 로드
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
base_model = AutoModelForMaskedLM.from_pretrained(MODEL_NAME)

print(f"✓ 토크나이저 로드 완료: {MODEL_NAME}")
print(f"✓ 기본 모델 로드 완료: {MODEL_NAME}")
print(f"   - Vocab size: {tokenizer.vocab_size:,}")
print(f"   - Model parameters: {sum(p.numel() for p in base_model.parameters()):,}")

class SPLADEModel(nn.Module):
    """
    SPLADE (Sparse Lexical and Expansion Model) 구현
    """
    def __init__(self, base_model, tokenizer):
        super().__init__()
        self.transformer = base_model
        self.tokenizer = tokenizer
        
        # SPLADE 특화 레이어
        hidden_size = base_model.config.hidden_size
        vocab_size = tokenizer.vocab_size
        
        # Log saturation activation
        self.log_activation = lambda x: torch.log1p(torch.relu(x))
    
    def forward(self, input_ids, attention_mask):
        """
        Forward pass
        Returns: sparse representation (batch_size, vocab_size)
        """
        # BERT 인코딩
        outputs = self.transformer(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True
        )
        
        # Logits (batch_size, seq_len, vocab_size)
        logits = outputs.logits
        
        # Max pooling over sequence length
        sparse_rep = torch.max(
            self.log_activation(logits) * attention_mask.unsqueeze(-1),
            dim=1
        ).values
        
        return sparse_rep

# SPLADE 모델 초기화
splade_model = SPLADEModel(base_model, tokenizer)
print("\n✓ SPLADE 모델 초기화 완료")

# GPU 사용 가능 시 이동
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
splade_model = splade_model.to(device)
print(f"✓ 모델을 {device}로 이동")

## 8. 학습 파라미터 설정

In [ ]:
# 학습 설정
BATCH_SIZE = 16
LEARNING_RATE = 2e-5
NUM_EPOCHS = 3
WARMUP_STEPS = 500
MAX_LENGTH = 128

# 손실 함수 가중치
FLOPS_REGULARIZATION = 1e-3  # L0 regularization
RANKING_LOSS_WEIGHT = 1.0

print("🎯 학습 하이퍼파라미터:")
print(f"  - Batch size: {BATCH_SIZE}")
print(f"  - Learning rate: {LEARNING_RATE}")
print(f"  - Epochs: {NUM_EPOCHS}")
print(f"  - Max length: {MAX_LENGTH}")
print(f"  - FLOPS regularization: {FLOPS_REGULARIZATION}")
print(f"  - Device: {device}")

## 9. 데이터 로더 준비

In [ ]:
class SPLADEDataset(torch.utils.data.Dataset):
    """
    SPLADE 학습용 데이터셋
    """
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        
        # 쿼리 토큰화
        query_encoded = self.tokenizer(
            item['query'],
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        # 문서 토큰화
        doc_encoded = self.tokenizer(
            item['document'],
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        return {
            'query_input_ids': query_encoded['input_ids'].squeeze(),
            'query_attention_mask': query_encoded['attention_mask'].squeeze(),
            'doc_input_ids': doc_encoded['input_ids'].squeeze(),
            'doc_attention_mask': doc_encoded['attention_mask'].squeeze(),
            'score': torch.tensor(item['score'], dtype=torch.float)
        }

# 데이터셋 분할 (train/val)
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(training_data, test_size=0.1, random_state=42)

train_dataset = SPLADEDataset(train_data, tokenizer, MAX_LENGTH)
val_dataset = SPLADEDataset(val_data, tokenizer, MAX_LENGTH)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"✓ 학습 데이터셋: {len(train_dataset):,} 샘플")
print(f"✓ 검증 데이터셋: {len(val_dataset):,} 샘플")
print(f"✓ 배치 수: {len(train_loader):,} (train), {len(val_loader):,} (val)")

## 10. 학습 루프 구현

In [ ]:
from torch.optim import AdamW
from torch.optim.lr_scheduler import get_linear_schedule_with_warmup
from tqdm.auto import tqdm

def splade_loss(query_rep, doc_rep, scores, flops_reg=1e-3):
    """
    SPLADE 손실 함수
    - Ranking loss (query-document similarity)
    - FLOPS regularization (L0 sparsity)
    """
    # Dot product similarity
    similarities = torch.sum(query_rep * doc_rep, dim=-1)
    
    # Ranking loss (MSE)
    ranking_loss = nn.functional.mse_loss(similarities, scores)
    
    # FLOPS regularization (encourage sparsity)
    query_flops = torch.sum(torch.abs(query_rep)) / query_rep.size(0)
    doc_flops = torch.sum(torch.abs(doc_rep)) / doc_rep.size(0)
    flops_loss = query_flops + doc_flops
    
    total_loss = ranking_loss + flops_reg * flops_loss
    
    return total_loss, ranking_loss, flops_loss

# 옵티마이저 설정
optimizer = AdamW(splade_model.parameters(), lr=LEARNING_RATE)
total_steps = len(train_loader) * NUM_EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=WARMUP_STEPS,
    num_training_steps=total_steps
)

print(f"✓ 옵티마이저 설정 완료 (AdamW, lr={LEARNING_RATE})")
print(f"✓ 스케줄러 설정 완료 (총 {total_steps:,} 스텝, warmup {WARMUP_STEPS:,} 스텝)")

# 학습 기록
training_history = {
    'train_loss': [],
    'val_loss': [],
    'train_ranking_loss': [],
    'train_flops_loss': []
}

def train_epoch(model, loader, optimizer, scheduler, device):
    """
    한 에폭 학습
    """
    model.train()
    total_loss = 0
    total_ranking_loss = 0
    total_flops_loss = 0
    
    progress_bar = tqdm(loader, desc="Training")
    
    for batch_idx, batch in enumerate(progress_bar):
        # 데이터 이동
        query_input_ids = batch['query_input_ids'].to(device)
        query_attention_mask = batch['query_attention_mask'].to(device)
        doc_input_ids = batch['doc_input_ids'].to(device)
        doc_attention_mask = batch['doc_attention_mask'].to(device)
        scores = batch['score'].to(device)
        
        # Forward pass
        query_rep = model(query_input_ids, query_attention_mask)
        doc_rep = model(doc_input_ids, doc_attention_mask)
        
        # 손실 계산
        loss, ranking_loss, flops_loss = splade_loss(
            query_rep, doc_rep, scores, FLOPS_REGULARIZATION
        )
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        # 기록
        total_loss += loss.item()
        total_ranking_loss += ranking_loss.item()
        total_flops_loss += flops_loss.item()
        
        # 진행 상황 업데이트
        progress_bar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'rank_loss': f'{ranking_loss.item():.4f}',
            'flops': f'{flops_loss.item():.2f}'
        })
    
    avg_loss = total_loss / len(loader)
    avg_ranking = total_ranking_loss / len(loader)
    avg_flops = total_flops_loss / len(loader)
    
    return avg_loss, avg_ranking, avg_flops

def evaluate(model, loader, device):
    """
    검증
    """
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        for batch in tqdm(loader, desc="Validation"):
            query_input_ids = batch['query_input_ids'].to(device)
            query_attention_mask = batch['query_attention_mask'].to(device)
            doc_input_ids = batch['doc_input_ids'].to(device)
            doc_attention_mask = batch['doc_attention_mask'].to(device)
            scores = batch['score'].to(device)
            
            query_rep = model(query_input_ids, query_attention_mask)
            doc_rep = model(doc_input_ids, doc_attention_mask)
            
            loss, _, _ = splade_loss(query_rep, doc_rep, scores, FLOPS_REGULARIZATION)
            total_loss += loss.item()
    
    return total_loss / len(loader)

print("\n" + "="*60)
print("🚀 학습 시작!")
print("="*60)

## 11. 모델 학습 실행

In [ ]:
# 학습 실행
best_val_loss = float('inf')

for epoch in range(NUM_EPOCHS):
    print(f"\n{'='*60}")
    print(f"Epoch {epoch + 1}/{NUM_EPOCHS}")
    print(f"{'='*60}")
    
    # 학습
    train_loss, train_ranking, train_flops = train_epoch(
        splade_model, train_loader, optimizer, scheduler, device
    )
    
    # 검증
    val_loss = evaluate(splade_model, val_loader, device)
    
    # 기록
    training_history['train_loss'].append(train_loss)
    training_history['val_loss'].append(val_loss)
    training_history['train_ranking_loss'].append(train_ranking)
    training_history['train_flops_loss'].append(train_flops)
    
    print(f"\n📊 Epoch {epoch + 1} 결과:")
    print(f"  Train Loss: {train_loss:.4f}")
    print(f"  Train Ranking Loss: {train_ranking:.4f}")
    print(f"  Train FLOPS Loss: {train_flops:.2f}")
    print(f"  Val Loss: {val_loss:.4f}")
    
    # 베스트 모델 저장
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save({
            'epoch': epoch,
            'model_state_dict': splade_model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
        }, 'best_korean_splade_model.pt')
        print(f"  ✓ 베스트 모델 저장! (val_loss: {val_loss:.4f})")

print("\n" + "="*60)
print("✅ 학습 완료!")
print("="*60)
print(f"최종 Best Validation Loss: {best_val_loss:.4f}")

## 12. 학습 결과 시각화

In [ ]:
import matplotlib.pyplot as plt

# 학습 곡선 시각화
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 전체 손실
axes[0, 0].plot(training_history['train_loss'], label='Train Loss', marker='o')
axes[0, 0].plot(training_history['val_loss'], label='Val Loss', marker='s')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].set_title('Training and Validation Loss')
axes[0, 0].legend()
axes[0, 0].grid(True)

# Ranking 손실
axes[0, 1].plot(training_history['train_ranking_loss'], label='Ranking Loss', marker='o', color='orange')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Ranking Loss')
axes[0, 1].set_title('Training Ranking Loss')
axes[0, 1].legend()
axes[0, 1].grid(True)

# FLOPS 손실
axes[1, 0].plot(training_history['train_flops_loss'], label='FLOPS Loss', marker='o', color='green')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('FLOPS Loss')
axes[1, 0].set_title('Training FLOPS Loss (Sparsity)')
axes[1, 0].legend()
axes[1, 0].grid(True)

# Loss 비교
x = np.arange(len(training_history['train_loss']))
width = 0.35
axes[1, 1].bar(x - width/2, training_history['train_loss'], width, label='Train', alpha=0.8)
axes[1, 1].bar(x + width/2, training_history['val_loss'], width, label='Val', alpha=0.8)
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Loss')
axes[1, 1].set_title('Train vs Val Loss Comparison')
axes[1, 1].set_xticks(x)
axes[1, 1].set_xticklabels([f'E{i+1}' for i in x])
axes[1, 1].legend()
axes[1, 1].grid(True, axis='y')

plt.tight_layout()
plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ 학습 곡선 저장: training_history.png")

## 13. 모델 평가 및 테스트

In [ ]:
# 베스트 모델 로드
checkpoint = torch.load('best_korean_splade_model.pt')
splade_model.load_state_dict(checkpoint['model_state_dict'])
splade_model.eval()

print("✓ 베스트 모델 로드 완료")
print(f"  - Epoch: {checkpoint['epoch'] + 1}")
print(f"  - Val Loss: {checkpoint['val_loss']:.4f}")

def encode_text(text, model, tokenizer, device, max_length=128):
    """
    텍스트를 SPLADE sparse vector로 인코딩
    """
    model.eval()
    
    # 토큰화
    encoded = tokenizer(
        text,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    
    input_ids = encoded['input_ids'].to(device)
    attention_mask = encoded['attention_mask'].to(device)
    
    # 인코딩
    with torch.no_grad():
        sparse_rep = model(input_ids, attention_mask)
    
    return sparse_rep.cpu().numpy()[0]

def get_top_tokens(sparse_vector, tokenizer, top_k=20):
    """
    Sparse vector에서 상위 토큰 추출
    """
    # 상위 k개 인덱스
    top_indices = np.argsort(sparse_vector)[-top_k:][::-1]
    top_values = sparse_vector[top_indices]
    
    # 토큰 변환
    top_tokens = [(tokenizer.decode([idx]), val) for idx, val in zip(top_indices, top_values) if val > 0]
    
    return top_tokens

# 테스트 쿼리
test_queries = [
    "인공지능 기반 검색 엔진",
    "딥러닝 모델 학습 방법",
    "한국어 자연어 처리 기술",
    "OpenSearch 벡터 검색",
    "머신러닝 알고리즘 최적화",
    "ChatGPT 같은 생성형 AI",
    "트랜스포머 아키텍처 설명",
]

print("\n" + "="*60)
print("🧪 테스트 쿼리 인코딩 결과")
print("="*60)

for query in test_queries:
    print(f"\n📝 쿼리: {query}")
    
    # 인코딩
    sparse_vec = encode_text(query, splade_model, tokenizer, device)
    
    # 희소성 계산
    non_zero_ratio = np.count_nonzero(sparse_vec) / len(sparse_vec) * 100
    l1_norm = np.sum(np.abs(sparse_vec))
    
    print(f"  희소성: {non_zero_ratio:.2f}% (non-zero: {np.count_nonzero(sparse_vec):,}/{len(sparse_vec):,})")
    print(f"  L1 Norm: {l1_norm:.2f}")
    
    # 상위 토큰
    top_tokens = get_top_tokens(sparse_vec, tokenizer, top_k=15)
    print(f"  상위 토큰:")
    for i, (token, value) in enumerate(top_tokens, 1):
        print(f"    {i:2d}. {token:15s} ({value:.4f})")

## 14. 모델 저장 및 내보내기

In [ ]:
# 모델 저장 디렉토리
OUTPUT_DIR = "./korean_splade_model"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 전체 모델 저장
torch.save(splade_model.state_dict(), f"{OUTPUT_DIR}/pytorch_model.bin")
tokenizer.save_pretrained(OUTPUT_DIR)

# 설정 파일 저장
config = {
    "model_name": MODEL_NAME,
    "vocab_size": tokenizer.vocab_size,
    "max_length": MAX_LENGTH,
    "training_samples": len(train_dataset),
    "epochs": NUM_EPOCHS,
    "best_val_loss": best_val_loss,
    "flops_regularization": FLOPS_REGULARIZATION,
    "learning_rate": LEARNING_RATE,
    "trained_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
}

with open(f"{OUTPUT_DIR}/config.json", 'w', encoding='utf-8') as f:
    json.dump(config, f, ensure_ascii=False, indent=2)

# 키워드 가중치 저장
weights_df = pd.DataFrame([
    {"keyword": k, "weight": v} 
    for k, v in sorted(keyword_weights.items(), key=lambda x: x[1], reverse=True)[:1000]
])
weights_df.to_csv(f"{OUTPUT_DIR}/keyword_weights.csv", index=False, encoding='utf-8')

# 동의어 사전 저장
with open(f"{OUTPUT_DIR}/synonyms.json", 'w', encoding='utf-8') as f:
    json.dump(BASE_SYNONYMS, f, ensure_ascii=False, indent=2)

print("✅ 모델 저장 완료!")
print(f"\n저장 위치: {OUTPUT_DIR}/")
print("\n저장된 파일:")
print("  - pytorch_model.bin: PyTorch 모델 가중치")
print("  - config.json: 모델 설정 및 학습 정보")
print("  - tokenizer files: 토크나이저 파일들")
print("  - keyword_weights.csv: 키워드 가중치")
print("  - synonyms.json: 동의어 사전")

## 15. OpenSearch 통합 가이드

In [ ]:
print("""
╔════════════════════════════════════════════════════════════════╗
║          OpenSearch Neural Sparse 모델 통합 가이드           ║
╚════════════════════════════════════════════════════════════════╝

1️⃣ 모델을 OpenSearch에 업로드:

   ```bash
   # 모델을 TorchScript로 변환
   python convert_to_torchscript.py --model ./korean_splade_model
   
   # OpenSearch에 모델 등록
   curl -XPOST "localhost:9200/_plugins/_ml/models/_upload" \
     -H 'Content-Type: application/json' \
     -d '{
       "name": "korean-splade-model",
       "version": "1.0",
       "model_format": "TORCH_SCRIPT",
       "model_config": {
         "model_type": "bert",
         "embedding_dimension": 768,
         "framework_type": "sentence_transformers"
       }
     }'
   ```

2️⃣ Neural Sparse Index 생성:

   ```json
   PUT /korean-neural-sparse-index
   {
     "settings": {
       "index": {
         "knn": true,
         "default_pipeline": "korean-neural-sparse-pipeline"
       }
     },
     "mappings": {
       "properties": {
         "content": { "type": "text" },
         "content_embedding": {
           "type": "rank_features"
         }
       }
     }
   }
   ```

3️⃣ Ingest Pipeline 설정:

   ```json
   PUT /_ingest/pipeline/korean-neural-sparse-pipeline
   {
     "description": "Korean Neural Sparse Encoding Pipeline",
     "processors": [
       {
         "sparse_encoding": {
           "model_id": "<model_id>",
           "field_map": {
             "content": "content_embedding"
           }
         }
       }
     ]
   }
   ```

4️⃣ Neural Sparse 검색:

   ```json
   POST /korean-neural-sparse-index/_search
   {
     "query": {
       "neural_sparse": {
         "content_embedding": {
           "query_text": "인공지능 기반 검색",
           "model_id": "<model_id>"
         }
       }
     }
   }
   ```

5️⃣ 하이브리드 검색 (BM25 + Neural Sparse):

   ```json
   POST /korean-neural-sparse-index/_search
   {
     "query": {
       "hybrid": {
         "queries": [
           {
             "match": {
               "content": "인공지능 검색"
             }
           },
           {
             "neural_sparse": {
               "content_embedding": {
                 "query_text": "인공지능 검색",
                 "model_id": "<model_id>"
               }
             }
           }
         ]
       }
     },
     "search_pipeline": {
       "phase_results_processors": [
         {
           "normalization-processor": {
             "normalization": {
               "technique": "min_max"
             },
             "combination": {
               "technique": "arithmetic_mean",
               "parameters": {
                 "weights": [0.3, 0.7]
               }
             }
           }
         }
       ]
     }
   }
   ```

📚 참고 자료:
  - OpenSearch Neural Search: https://opensearch.org/docs/latest/search-plugins/neural-search/
  - Neural Sparse Search: https://opensearch.org/docs/latest/search-plugins/neural-sparse-search/
  - Hybrid Search: https://opensearch.org/docs/latest/search-plugins/hybrid-search/

""")

## 16. 성능 벤치마크

In [ ]:
import time

# 벤치마크 테스트
benchmark_queries = [
    "머신러닝",
    "인공지능 딥러닝",
    "한국어 자연어 처리 기술 연구",
    "OpenSearch 벡터 검색 엔진 최적화 방법",
]

print("\n" + "="*60)
print("⚡ 성능 벤치마크")
print("="*60)

for query in benchmark_queries:
    # 시간 측정
    start_time = time.time()
    sparse_vec = encode_text(query, splade_model, tokenizer, device)
    end_time = time.time()
    
    encoding_time = (end_time - start_time) * 1000  # ms
    non_zero_count = np.count_nonzero(sparse_vec)
    sparsity = (1 - non_zero_count / len(sparse_vec)) * 100
    
    print(f"\n쿼리: {query}")
    print(f"  인코딩 시간: {encoding_time:.2f}ms")
    print(f"  희소성: {sparsity:.2f}% (non-zero: {non_zero_count}/{len(sparse_vec)})")
    print(f"  벡터 크기: {len(sparse_vec):,} dimensions")

print("\n" + "="*60)
print("✅ 벤치마크 완료!")
print("="*60)

## 17. 요약 및 다음 단계

### 🎉 완료된 작업

1. ✅ 한국어 공개 데이터셋 수집 (KLUE, KorQuAD, Wikipedia 등)
2. ✅ Mecab 형태소 분석기를 통한 키워드 추출
3. ✅ 동의어 사전 구축 및 트렌드 키워드 가중치 적용
4. ✅ SPLADE 모델 파인튜닝 (한국어 BERT 기반)
5. ✅ 모델 평가 및 벤치마크
6. ✅ OpenSearch 통합 가이드 제공

### 📈 성능 개선 포인트

- **희소성**: FLOPS regularization을 통해 효율적인 sparse representation 달성
- **트렌드 반영**: 2024-2025 최신 키워드(LLM, GPT, RAG 등)에 높은 가중치 부여
- **동의어 처리**: 한국어 특성을 고려한 동의어 확장
- **형태소 분석**: Mecab을 통한 정확한 한국어 토큰화

### 🚀 다음 단계

1. **더 많은 데이터로 학습**: AI Hub, NIKL 등 추가 데이터셋 활용
2. **Hard Negative Mining**: 어려운 negative 샘플로 학습 강화
3. **Cross-encoder Reranking**: 재순위화 모델 추가
4. **Multi-task Learning**: NER, QA 등 다양한 태스크 동시 학습
5. **A/B 테스팅**: OpenSearch에서 기존 모델과 성능 비교
6. **지속적 업데이트**: 최신 트렌드 키워드 주기적 업데이트

### 📝 참고

이 노트북은 한국어 neural sparse 검색의 기초를 다지는 출발점입니다. 
실제 프로덕션 환경에서는 더 많은 데이터, 더 긴 학습 시간, 그리고 하이퍼파라미터 튜닝이 필요합니다.
"""